In [3]:
from bayesian_optimization import BayesianOptimization
import pandas as pd 
from botorch.test_functions import Ackley
from botorch.test_functions import Levy 

# Variables that determine the size of the dataset
N_INITIAL = 100
EPOCHS = 10
BATCH_SIZE = 10
DIM = 10
# Bounds are 10 for Levy and 32 for Ackley 

optimizer = BayesianOptimization(fun=Ackley(dim=DIM, negate=True), 
                                 batch_size=BATCH_SIZE, 
                                 dim=DIM, 
                                 epochs=EPOCHS, 
                                 n_init=N_INITIAL, 
                                 lower_bound=-32.176,
                                 upper_bound=32.176,
                                 seed=5,
                                 acqf_type='qUCB')

x_max, y_max = optimizer.run()
data = optimizer.get_data()
full = optimizer.format(data, dim=DIM, n_init=N_INITIAL, batch_size=BATCH_SIZE, epochs=EPOCHS)
display(full)

,Batch,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,y
0,0.0,-1.268344,-24.630157,-24.197015,-9.895261,-18.438448,3.098315,-4.773726,27.769696,8.628926,-4.579019,-20.991023
1,0.0,11.050301,11.118967,16.558254,23.985074,0.761052,-20.643286,25.763249,-29.443123,-19.246696,26.092024,-21.243080
2,0.0,31.271793,-8.915932,-5.679147,-28.166922,19.969438,-0.032730,-30.915526,-0.165285,-14.733306,10.061175,-20.617730
3,0.0,-20.763086,27.831074,13.836867,2.068094,-1.302439,23.736404,3.641597,7.022896,21.208903,-23.626410,-20.943148
4,0.0,-25.446474,-6.089227,30.476108,12.502565,-14.907971,10.281443,-21.592235,-12.709945,-7.720115,18.108712,-21.307879
...,...,...,...,...,...,...,...,...,...,...,...,...
195,10.0,-16.379544,10.095375,23.928198,-28.105722,25.956668,29.291693,-19.320117,-16.280629,-21.818231,18.779103,-21.180056
196,10.0,1.034289,0.868871,-0.039757,1.500435,-7.930238,-3.852081,0.265403,-1.015344,3.685729,-5.496303,-11.602571
197,10.0,7.634781,29.202910,-11.556451,-5.025766,2.501414,-13.996777,-26.765416,1.962671,-1.945030,-21.288447,-20.688009
198,10.0,20.935442,-18.630749,-31.235498,-23.810601,11.476616,10.227921,23.056145,3.032141,-13.426515,-1.248540,-21.093680


In [13]:
import pandas as pd
cols = ['Batch','y']
df = full[cols]


y_0 = df[df['Batch'] == 0.0][['y']]

# Reset the index (optional, to get a clean index starting from 0)
# y_0 = y_0.reset_index(drop=True)

print(y_0)


            y
0  -20.991023
1  -21.243080
2  -20.617730
3  -20.943148
4  -21.307879
..        ...
95 -21.423673
96 -21.129790
97 -21.140323
98 -20.986738
99 -21.172051

[100 rows x 1 columns]


In [7]:
import numpy as np

def bootstrap_diff_max(X, Y, n_bootstrap=10000):
    np.random.seed(0)
    max_diffs = []
    for _ in range(n_bootstrap):
        X_resample = np.random.choice(X, size=len(X), replace=True)
        Y_resample = np.random.choice(Y, size=len(Y), replace=True)
        max_diff = np.max(X_resample) - np.max(Y_resample)
        max_diffs.append(max_diff)
    return np.array(max_diffs)

# Sample data
X = np.random.normal(10, 5, 100)  # Replace with your data
Y = np.random.normal(12, 5, 100)  # Replace with your data
print(X.shape,Y)
# Bootstrap method
max_diffs_bootstrap = bootstrap_diff_max(X, Y)
obs_diff = np.max(X) - np.max(Y)

# Confidence interval
lower_bound_bootstrap = np.percentile(max_diffs_bootstrap, 2.5)
upper_bound_bootstrap = np.percentile(max_diffs_bootstrap, 97.5)

print(f"Bootstrap Observed difference of maxima: {obs_diff}")
print(f"Bootstrap 95% Confidence interval: [{lower_bound_bootstrap}, {upper_bound_bootstrap}]")

(100,) [14.00302964 13.33234702  8.83697698 14.83799489 10.61643466 12.95624483
 11.32477743 13.34966646  7.63591721 13.23926794  8.14818748 11.97457334
 11.86943172 10.74621642  9.01436598  4.42570815 14.77242682  0.37546632
 12.88741441 21.13040823 16.92857936 14.98388762 21.65543913 13.81506922
 14.10757865 12.52171427  2.18380401  0.21567805 15.26731805 21.52886234
 10.17045288 14.28136188  7.41187752  9.88449633  9.87892256  5.07737545
 18.68464756 17.70974916 12.66915657 13.43160797  3.15234204 13.93332753
  7.48463858  9.27536004 14.69511839 14.60888822 12.04451313 11.88683888
  7.13955616 14.72053995 13.19677404 10.377187   12.98737196  8.51205275
 18.465986    6.10944008 14.71901799 17.41201261 12.75917887  7.80547452
 14.02244076  9.56718169 11.9909538   7.18942011  0.55233943  6.2818943
 15.99019026 19.47860382 18.54226498 23.89401077  9.25289047  9.40407493
 14.73677027 20.17710613 19.1363449  12.77934344 13.86493255  1.49414113
 14.24837411  9.16091274  0.42758215 12.38396